# Classificação dos tipos de cogumelo

O objetivo desse projeto é conseguir classificar um cogumelo como comestível ou venenoso.

- Esther Dagir
- Gustavo Katz
- Lila Habda
- Rafael Oncins

### Importando bibliotecas e fazendo a leitura do Dataframe

In [ ]:
import pandas as pd
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, probplot
from sklearn import tree

#leitura do Dataframe
df = pd.read_csv('mushrooms.csv')

In [ ]:
# Dataframe dos cogumelos
df.head(5)

## Análise exploratória

A ideia nessa análise é avaliar a influência de cada categoria na determinação de um cogumelo como comestível ou venenoso. Para isso, vamos criar gráficos que analisam a frequência de determinada característica em cada tido de cogumelo, conforme cada categoria (coluna).

In [ ]:
# Para construir um gráfico para cada coluna, vamos dividir em três células a fim de facilitarr a visualização
# Gráficos que demonstram as características dos cogumelos conforme sua classificação
# Crosstab entre a classe (comestível ou venenoso) e as colunas do dataframe 

lista = ['cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size']

for i in lista:
    cross = pd.crosstab(df["class"], df[i], normalize="index")
    plot = cross.plot(kind='bar', stacked=True,title=f'Class por {i}')
    plt.ylabel('Freq.Relativa (em%)')
    ax = plt.subplot(111)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# Gráficos que demonstram as características dos cogumelos conforme sua classificação
# Crosstab entre a classe (comestível ou venenoso) e as colunas do dataframe 

lista = ['stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring']

for i in lista:
    cross = pd.crosstab(df["class"], df[i], normalize="index")
    plot = cross.plot(kind='bar', stacked=True,title=f'Class por {i}')
    plt.ylabel('Freq.Relativa (em%)')
    ax = plt.subplot(111)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
# Gráficos que demonstram as características dos cogumelos conforme sua classificação
# Crosstab entre a classe (comestível ou venenoso) e as colunas do dataframe 

lista = ['veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat']

for i in lista:
    cross = pd.crosstab(df["class"], df[i], normalize="index")
    plot = cross.plot(kind='bar', stacked=True,title=f'Class por {i}')
    plt.ylabel('Freq.Relativa (em%)')
    ax = plt.subplot(111)
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

## Análise inicial
A partir da análise dos gráficos, foi possível identificar as categorias que mais influenciam a determinação de um cogumelo como comestível ou venenoso. Dessa maneira, filtramos algumas categorias:

odor, stalk-color-above-ring, ring-type, spore-print-color, population, habitat

Justificativa pela escolha das categorias:
- Odor: Pode-se perceber que pelo gráfico odor X class, a primeira tem uma grande influência na segunda já que há uma grande distinção das características e o odor do tipo "n" parece ser um fator de relevância para a classe "comestível".


- Stalk-color-above-ring: Apesar de o tipo "p" e "w" apresentrem relevância nas duas classes, pode-se notar uma boa diferenciabilidade entre elas nos outros tipos, visto que essas, por sua peculiaridade, farão uma distinção de comestível e não comestível.


- Ring-type: Nesse gráfico, é possível observar principalmente a influência da característica "l" muito presente nos cogumelos venenosos.


- Spore-print-color:Nesse gráfico, embora ambas as classes tenham essas características, a classe "comestível" tem uma alta frenquência dos fatores "n" e "k", enquanto a classe do tipo "venenoso" tem uma alta frenquência dos fatores "w" e "h".


- Population: Assim como no item 2, os tipos "v", "s" e "y" apresentam grande presença nos tipos comestível e não comestível. No entanto, os demais tipos tem uma maior diferença, sendo esse em maioria no comestível, o que permite diferenciar tais classes.


- Habitat: No gráfico do habitat, aparentemente há uma semelhança entre os fatores para ambas classes, porém as quantidades são diferentes para cada classe, principalmente do tipo "p".

A partir desse ponto, nós vamos utilizar apenas essas colunas como referências para o nosso modelo. Portanto, criamos um novo dataframe:

In [ ]:
# Separando somente as colunas que serão utilizadas na classificação
cl_utilizadas = df.loc[:,['odor','stalk-color-above-ring','ring-type','spore-print-color','population','habitat']]
cl_utilizadas

In [ ]:
# Trocando as variáveis letras para números, para conseguir utilizar essa biblioteca
x = pd.get_dummies(cl_utilizadas)

df['class'] = df['class'].replace('e', 0)
df['class'] = df['class'].replace('p', 1)
y = df['class']

## Árvore de decisão

A seguir, montamos uma árvore de decisão com as categorias selecionadas a fim de observar o seu comportamento. A árvore de decisão é uma maneira de observar um processo de escolhas conforme a probabilidade de algum evento acontecer. Com isso, a nossa árvore irá demonstrar quais são as características mais significativas para determinar se um cogumelo é ou não comestível.

In [ ]:
# árvore de decisão
# uma maneira de classificar um cogumelo como comestível ou venenoso, com base nas características selecionadas acima
clf = tree.DecisionTreeClassifier(random_state = 0)
clf = clf.fit(x, df['class'])

fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf, feature_names=x.columns, filled=True)

A árvore de decisão facilita a compreensão das principais características que influenciam na classificação dos cogumelos. No caso, podemos observar que o odor do tipo "n" parece ser um fator significante nesse processo de seleção. Além dele, o odor do tipo "l" e o spore-print-color do tipo "r" aparentam ser fatores significantes também.

## Implementando os modelos de predição
Para começar, vamos separar os dados em uma coluna de treinamento e uma de teste. Essas colunas serão usadas para ambos os modelos de predição. 
A planilha de testes corresponde a 20% dos dados. 

In [ ]:
# Separando os dados em treinamento e teste
from sklearn.model_selection import train_test_split

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.2)

## Primeiro modelo

Nesse primeiro modelo de predição, utilizamos as categorias que mais influenciam para criar um classificador. Assim, a biblioteca sklearn utilizada separa os dados em uma parte de treinamento e outra de teste. Ao se ajustar com o treinamento, o classificador consegue avaliar a parte de teste e trazer resultados precisos.

In [ ]:
# Aplicando o classificador
from sklearn.ensemble import ExtraTreesClassifier

modelo1 = ExtraTreesClassifier(n_estimators = 100) 
M1 = modelo1.fit(x_treino, y_treino) # modelo ajustado conforme a planilha de treinamento

resultado_treino = M1.score(x_treino, y_treino) # resutado da aplicação do classificador na própria planilha de treinamento
resultado_teste = M1.score(x_teste, y_teste) # resultado obtido com a planilha teste

print(f'Acurácia do treino:{resultado_treino}')
print(f'Acurácia do teste: {resultado_teste}')

## Segundo modelo

No segundo modelo, utilizamos a mesma biblioteca sklearn, e os mesmos dados de treinamento e teste, a fim de ser uma melhor maneira de comparar os resultados no final. Nesse caso, utilizamos a técnica da regressão logística para gerar um classificador.

In [ ]:
from sklearn.linear_model import LogisticRegression

modelo2= LogisticRegression(fit_intercept = False)
M2 = modelo2.fit(x_treino, y_treino)

resultado_treino2 = M2.score(x_treino, y_treino) # resutado da aplicação do classificador na própria planilha de treinamento
resultado_teste2 = M2.score(x_teste, y_teste) # resultado obtido com a planilha teste

print(f'Acurácia do treino:{resultado_treino2}')
print(f'Acurácia do teste: {resultado_teste2}')

In [ ]:
coeficientes = pd.DataFrame(data={'coeficientes':M2.coef_[0],'treinamento':x_treino.columns})
coeficientes.sort_values('coeficientes')

- Explicação melhor sobre árvore de decisão( o que é, link de referência)
- Explicação do porquê escolher aquelas categorias
- Verificar e validar modelos
- Relatório (explicação do passo a passo, descrever tudo, conclusão)